In [122]:
import re
def split(txt):
    txt = txt.split("$", 2)
    if txt:
        txt = str("$" + str(txt[1]))
    return txt

In [129]:
from selectorlib import Extractor
import requests 
import json
from time import sleep
from bs4 import BeautifulSoup

# Create an Extractor by reading from the YAML file
# e = Extractor.from_yaml_file('selectors.yml')

def scrape(url):  

    headers = {
        'dnt': '1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.61 Safari/537.36',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-user': '?1',
        'sec-fetch-dest': 'document',
        'referer': 'https://www.amazon.com/',
        'accept-language': 'en-GB,en-US;q=0.9,en;q=0.8',
    }

    print("Downloading %s"%url)
    r = requests.get(url, headers=headers)
    print(r.status_code)
    if r.status_code > 500:
        if "To discuss automated access to Amazon data please contact" in r.text:
            print("Page %s was blocked by Amazon. Please try using better proxies\n"%url)
        else:
            print("Page %s must have been blocked by Amazon as the status code was %d"%(url,r.status_code))
        return None
    
    soup = BeautifulSoup(r.text, "html.parser")

    name = None
    selector1 = "#productTitle"
    el = soup.select_one(selector1)
    if el:
        name = el.get_text()
    else:
        selector2 = "#alternateProductName"
        el = soup.select_one(selector2)
        if el:
            name = el.get_text()

    price = None
    selectors = [
        "span#price_inside_buybox",
        "#alternative_price",
        "span.a-price.aok-align-center.reinventPricePriceToPayMargin.priceToPay"]

    for selector in selectors:
        el = soup.select_one(selector)
        if el:
            price = el.get_text()
            price = split(price)
            break
    int_price = float((price.replace("$", "")).replace(" ", ""))
            
    original_price = None
    selectors = [
        "td.a-span12.a-color-secondary span.a-price",
        "span.a-price[data-a-strike='true']"]  

    for selector in selectors:
        el = soup.select_one(selector)
        if el:
            original_price = el.get_text()
            original_price = split(original_price)
            break
        
    if original_price:
        int_original = float((original_price.replace("$", "")).replace(" ", ""))
        discount = str(round((1 - int_price / int_original) * 100)) + "%"
    else:
        original_price = price
        discount = None

    data = {
       "name": name,
       "price": price,
       "discount": discount,
       "original price": original_price,
    }
    
    return data
      
with open("urls.txt") as f, open("output.jsonl", "w") as outfile:    
    for url in f:
        data = scrape(url.strip())  
          
        if data:     
           json.dump(data, outfile)           
           outfile.write("\n")  
           sleep(5)

200
200
200
200
200
200
